In [1]:
import time
from joblib import Parallel, delayed
from scipy.io import loadmat 
import numpy as np
from pandas import read_csv
from multiprocessing import Process, Queue
from math import floor


In [2]:

# filename = 'Mug128_singlepixcam.mat'
filename = 'regression_dataset.mat'
array = loadmat(filename)

X = np.array(array['train'])
Y = np.array(array['train_y'])
nr,nc = X.shape
# print X.shape
beta = 1;  # For Squared Loss
lamda = 0.216;

#Normalization-1
minimum = X.min(0);
maximum = X.max(0);

for i in np.arange(nc):
    if(maximum[i]!=minimum[i]):
        X[:,i] = (X[:,i]-minimum[i]) / (maximum[i]-minimum[i]);
    else:
        X[:,i] = np.ones(nr)

#Normalization-2
X = X - X.mean(axis=0)
nrm = np.linalg.norm(X,axis = 0);

for i in np.arange(nc):
	if nrm[i]>0:
		X[:,i] = X[:,i]/nrm[i]

Data = X
X = np.concatenate((X,-X), axis = 1)
nc = 2*nc;
# print nc,X.shape
# print nc
# sys.exit(0)

w = np.zeros(nc,dtype = np.float64).reshape(nc,1) +1
# w = np.random.rand(nc,1)

def func(index):
	# print index
	t1 = np.dot(X,w)
	t2 = t1- Y

	g = np.dot(X[:,index].T,t2) + lamda; #why nr??
	w[index] = w[index] -   0.2*g/beta; # alpha
	# print w[index]
	if w[index]<0:
		w[index] = 0;

	return w[index]
    
for i in np.arange(100):
    for k in np.arange(0,nc,2):
        with Parallel(n_jobs=10) as parallel:
            results = parallel(delayed(func)(j)for j in np.arange(k,k+2) )
        w[k:k+2] = results
    
    print (i)    
    temp = nc/2
    z1 = np.subtract(w[temp:nc] , w[0:temp])
    z1 = z1.reshape(temp,1)

    z2 = np.subtract(w[0:temp],w[temp:nc])
    z2 = z2.reshape(temp,1)
    # pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
    pred = np.dot(Data,z1)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z1 w2-w1 MSE Error:%f' % MSE)

    pred = np.dot(Data,z2)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z2  w1-w2  MSE Error:%f' % MSE)
            
    print (np.linalg.norm(w))


# z1 w2-w1 MSE Error:249.490688
# z2  w1-w2  MSE Error:124.301825
# 124.034832325
# 3
# z1 w2-w1 MSE Error:269.551688
# z2  w1-w2  MSE Error:118.262161
# 149.896281743
# 4
# z1 w2-w1 MSE Error:286.597020
# z2  w1-w2  MSE Error:114.373997
# 170.639428595
# 5
# z1 w2-w1 MSE Error:300.883147
# z2  w1-w2  MSE Error:111.870165
# 187.27819572
# 6
# z1 w2-w1 MSE Error:312.737374
# z2  w1-w2  MSE Error:110.257262
# 200.625449426
# 7
# z1 w2-w1 MSE Error:322.500341
# z2  w1-w2  MSE Error:109.217888
# 211.332899538
# 8
# z1 w2-w1 MSE Error:330.495466
# z2  w1-w2  MSE Error:108.547822
# 219.923085623
# 9
# z1 w2-w1 MSE Error:337.014411
# z2  w1-w2  MSE Error:108.115633
# 226.815009302
# 10
# z1 w2-w1 MSE Error:342.311848
# z2  w1-w2  MSE Error:107.836716
# 232.344673859
# 11
# z1 w2-w1 MSE Error:346.605379
# z2  w1-w2  MSE Error:107.656596
# 236.781543785
# 12
# z1 w2-w1 MSE Error:350.078112
# z2  w1-w2  MSE Error:107.540186
# 240.341735794
# 13
# z1 w2-w1 MSE Error:352.882435
# z2  w1-w2  MSE Error:107.464881
# 243.198591335
# 14
# z1 w2-w1 MSE Error:355.144128
# z2  w1-w2  MSE Error:107.416111
# 245.491151196
# 15
# z1 w2-w1 MSE Error:356.966366
# z2  w1-w2  MSE Error:107.384484
# 247.330949118
# 16
# z1 w2-w1 MSE Error:358.433379
# z2  w1-w2  MSE Error:107.363941
# 248.807458352
# 17
# z1 w2-w1 MSE Error:359.613683
# z2  w1-w2  MSE Error:107.350570
# 249.992458625
# 18
# z1 w2-w1 MSE Error:360.562848
# z2  w1-w2  MSE Error:107.341847
# 250.943537787
# 19
# z1 w2-w1 MSE Error:361.325845
# z2  w1-w2  MSE Error:107.336140
# 251.706899832
# 20
# z1 w2-w1 MSE Error:361.939006
# z2  w1-w2  MSE Error:107.332393
# 252.319616824
# 21
# z1 w2-w1 MSE Error:362.431641
# z2  w1-w2  MSE Error:107.329923
# 252.811435003
# 22
# z1 w2-w1 MSE Error:362.827371
# z2  w1-w2  MSE Error:107.328287
# 253.206223393
# 23
# z1 w2-w1 MSE Error:363.145213
# z2  w1-w2  MSE Error:107.327196
# 253.523135754
# 24
# z1 w2-w1 MSE Error:363.400473
# z2  w1-w2  MSE Error:107.326465
# 253.777542658
# 25
# z1 w2-w1 MSE Error:363.605456
# z2  w1-w2  MSE Error:107.325971
# 253.981779183
# 26
# z1 w2-w1 MSE Error:363.770057
# z2  w1-w2  MSE Error:107.325634
# 254.145744749
# 27
# z1 w2-w1 MSE Error:363.902226
# z2  w1-w2  MSE Error:107.325401
# 254.277384341
# 28
# z1 w2-w1 MSE Error:364.008351
# z2  w1-w2  MSE Error:107.325240
# 254.383074601
# 29
# z1 w2-w1 MSE Error:364.093564
# z2  w1-w2  MSE Error:107.325126
# 254.467933595
# 30
# z1 w2-w1 MSE Error:364.161984
# z2  w1-w2  MSE Error:107.325045
# 254.536069371
# 31
# z1 w2-w1 MSE Error:364.216921
# z2  w1-w2  MSE Error:107.324987
# 254.590779389
# 32
# z1 w2-w1 MSE Error:364.261034
# z2  w1-w2  MSE Error:107.324944
# 254.634710558
# 33
# z1 w2-w1 MSE Error:364.296454
# z2  w1-w2  MSE Error:107.324913
# 254.669987654
# 34
# z1 w2-w1 MSE Error:364.324896
# z2  w1-w2  MSE Error:107.324889
# 254.698316377
# 35
# z1 w2-w1 MSE Error:364.347734
# z2  w1-w2  MSE Error:107.324871
# 254.72106605
# 36
# z1 w2-w1 MSE Error:364.366074
# z2  w1-w2  MSE Error:107.324858
# 254.739335995
# 37
# z1 w2-w1 MSE Error:364.380801
# z2  w1-w2  MSE Error:107.324847
# 254.7540088
# 38
# z1 w2-w1 MSE Error:364.392628
# z2  w1-w2  MSE Error:107.324839
# 254.765793074
# 39
# z1 w2-w1 MSE Error:364.402126
# z2  w1-w2  MSE Error:107.324833
# 254.775257757
# 40
# z1 w2-w1 MSE Error:364.409754
# z2  w1-w2  MSE Error:107.324828
# 254.782859671
# 41
# z1 w2-w1 MSE Error:364.415880
# z2  w1-w2  MSE Error:107.324824
# 254.788965622
# 42
# z1 w2-w1 MSE Error:364.420800
# z2  w1-w2  MSE Error:107.324821
# 254.793870146
# 43
# z1 w2-w1 MSE Error:364.424752
# z2  w1-w2  MSE Error:107.324819
# 254.797809761
# 44
# z1 w2-w1 MSE Error:364.427926
# z2  w1-w2  MSE Error:107.324817
# 254.800974398
# 45
# z1 w2-w1 MSE Error:364.430475
# z2  w1-w2  MSE Error:107.324815
# 254.803516583
# 46
# z1 w2-w1 MSE Error:364.432523
# z2  w1-w2  MSE Error:107.324814
# 254.805558805
# 47
# z1 w2-w1 MSE Error:364.434168
# z2  w1-w2  MSE Error:107.324813
# 254.80719944
# 48
# z1 w2-w1 MSE Error:364.435489
# z2  w1-w2  MSE Error:107.324812
# 254.808517496
# 49
# z1 w2-w1 MSE Error:364.436551
# z2  w1-w2  MSE Error:107.324812
# 254.809576428
# 50
# z1 w2-w1 MSE Error:364.437403
# z2  w1-w2  MSE Error:107.324811
# 254.810427203
# 51
# z1 w2-w1 MSE Error:364.438088
# z2  w1-w2  MSE Error:107.324811
# 254.811110759
# 52
# z1 w2-w1 MSE Error:364.438639
# z2  w1-w2  MSE Error:107.324810
# 254.811659978
# 53
# z1 w2-w1 MSE Error:364.439081
# z2  w1-w2  MSE Error:107.324810
# 254.812101274
# 54
# z1 w2-w1 MSE Error:364.439436
# z2  w1-w2  MSE Error:107.324810
# 254.812455863
# 55
# z1 w2-w1 MSE Error:364.439722
# z2  w1-w2  MSE Error:107.324810
# 254.81274079
# 56
# z1 w2-w1 MSE Error:364.439951
# z2  w1-w2  MSE Error:107.324809
# 254.812969747
# 57
# z1 w2-w1 MSE Error:364.440135
# z2  w1-w2  MSE Error:107.324809
# 254.813153733
# 58
# z1 w2-w1 MSE Error:364.440283
# z2  w1-w2  MSE Error:107.324809
# 254.813301585
# 59
# z1 w2-w1 MSE Error:364.440402
# z2  w1-w2  MSE Error:107.324809
# 254.813420404
# 60
# z1 w2-w1 MSE Error:364.440498
# z2  w1-w2  MSE Error:107.324809
# 254.813515892
# 61
# z1 w2-w1 MSE Error:364.440575
# z2  w1-w2  MSE Error:107.324809
# 254.813592633
# 62
# z1 w2-w1 MSE Error:364.440636
# z2  w1-w2  MSE Error:107.324809
# 254.813654309
# 63
# z1 w2-w1 MSE Error:364.440686
# z2  w1-w2  MSE Error:107.324809
# 254.813703879
# 64
# z1 w2-w1 MSE Error:364.440726
# z2  w1-w2  MSE Error:107.324809
# 254.81374372
# 65
# z1 w2-w1 MSE Error:364.440758
# z2  w1-w2  MSE Error:107.324809
# 254.813775742
# 66
# z1 w2-w1 MSE Error:364.440784
# z2  w1-w2  MSE Error:107.324809
# 254.81380148
# 67
# z1 w2-w1 MSE Error:364.440804
# z2  w1-w2  MSE Error:107.324809
# 254.813822169
# 68
# z1 w2-w1 MSE Error:364.440821
# z2  w1-w2  MSE Error:107.324809
# 254.813838798
# 69
# z1 w2-w1 MSE Error:364.440834
# z2  w1-w2  MSE Error:107.324809
# 254.813852166
# 70
# z1 w2-w1 MSE Error:364.440845
# z2  w1-w2  MSE Error:107.324809
# 254.813862911
# 71
# z1 w2-w1 MSE Error:364.440854
# z2  w1-w2  MSE Error:107.324809
# 254.813871549
# 72
# z1 w2-w1 MSE Error:364.440861
# z2  w1-w2  MSE Error:107.324809
# 254.813878493
# 73
# z1 w2-w1 MSE Error:364.440866
# z2  w1-w2  MSE Error:107.324809
# 254.813884075
# 74
# z1 w2-w1 MSE Error:364.440871
# z2  w1-w2  MSE Error:107.324809
# 254.813888563
# 75
# z1 w2-w1 MSE Error:364.440874
# z2  w1-w2  MSE Error:107.324809
# 254.813892171
# 76
# z1 w2-w1 MSE Error:364.440877
# z2  w1-w2  MSE Error:107.324809
# 254.813895072
# 77
# z1 w2-w1 MSE Error:364.440879
# z2  w1-w2  MSE Error:107.324809
# 254.813897404
# 78
# z1 w2-w1 MSE Error:364.440881
# z2  w1-w2  MSE Error:107.324809
# 254.813899279
# 79
# z1 w2-w1 MSE Error:364.440883
# z2  w1-w2  MSE Error:107.324809
# 254.813900786
# 80
# z1 w2-w1 MSE Error:364.440884
# z2  w1-w2  MSE Error:107.324809
# 254.813901998
# 81
# z1 w2-w1 MSE Error:364.440885
# z2  w1-w2  MSE Error:107.324809
# 254.813902973
# 82
# z1 w2-w1 MSE Error:364.440886
# z2  w1-w2  MSE Error:107.324809
# 254.813903757
# 83
# z1 w2-w1 MSE Error:364.440886
# z2  w1-w2  MSE Error:107.324809
# 254.813904387
# 84
# z1 w2-w1 MSE Error:364.440887
# z2  w1-w2  MSE Error:107.324809
# 254.813904893
# 85
# z1 w2-w1 MSE Error:364.440887
# z2  w1-w2  MSE Error:107.324809
# 254.813905301
# 86
# z1 w2-w1 MSE Error:364.440888
# z2  w1-w2  MSE Error:107.324809
# 254.813905628
# 87
# z1 w2-w1 MSE Error:364.440888
# z2  w1-w2  MSE Error:107.324809
# 254.813905892
# 88
# z1 w2-w1 MSE Error:364.440888
# z2  w1-w2  MSE Error:107.324809
# 254.813906104
# 89
# z1 w2-w1 MSE Error:364.440888
# z2  w1-w2  MSE Error:107.324809
# 254.813906274
# 90
# z1 w2-w1 MSE Error:364.440888
# z2  w1-w2  MSE Error:107.324809
# 254.813906411
# 91
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906521
# 92
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.81390661
# 93
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906681
# 94
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906738
# 95
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906785
# 96
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906822
# 97
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906851
# 98
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906875
# 99
# z1 w2-w1 MSE Error:364.440889
# z2  w1-w2  MSE Error:107.324809
# 254.813906895


0
z1 w2-w1 MSE Error:200.333703
z2  w1-w2  MSE Error:148.270604
51.5868181525


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:65: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1
z1 w2-w1 MSE Error:226.330306
z2  w1-w2  MSE Error:133.686052
91.7904275112
2
z1 w2-w1 MSE Error:249.490688
z2  w1-w2  MSE Error:124.301825
124.034832325
3
z1 w2-w1 MSE Error:269.551688
z2  w1-w2  MSE Error:118.262161
149.896281743
4
z1 w2-w1 MSE Error:286.597020
z2  w1-w2  MSE Error:114.373997
170.639428595
5
z1 w2-w1 MSE Error:300.883147
z2  w1-w2  MSE Error:111.870165
187.27819572
6
z1 w2-w1 MSE Error:312.737374
z2  w1-w2  MSE Error:110.257262
200.625449426
7
z1 w2-w1 MSE Error:322.500341
z2  w1-w2  MSE Error:109.217888
211.332899538
8
z1 w2-w1 MSE Error:330.495466
z2  w1-w2  MSE Error:108.547822
219.923085623
9
z1 w2-w1 MSE Error:337.014411
z2  w1-w2  MSE Error:108.115633
226.815009302
10
z1 w2-w1 MSE Error:342.311848
z2  w1-w2  MSE Error:107.836716
232.344673859
11
z1 w2-w1 MSE Error:346.605379
z2  w1-w2  MSE Error:107.656596
236.781543785
12
z1 w2-w1 MSE Error:350.078112
z2  w1-w2  MSE Error:107.540186
240.341735794
13
z1 w2-w1 MSE Error:352.882435
z2  w1-w2  MSE Error:107.4648

In [11]:
temp = nc/2
z = np.subtract(w[0:temp],w[temp:nc])
z = z.reshape(temp,1)
# pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
pred = np.dot(Data,z)
differ = (pred-Y)
MSE = np.mean( np.square(differ) );
print ('MSE Error:%f' % MSE)

ts = np.array(array['test'])
ts_y = np.array(array['test_y'])
pred = np.dot(ts, z)
differ = (pred.T-ts_y)
MSE = np.mean( np.square(differ) );
print ('MSE Error:%f' % MSE)



MSE Error:107.324809
MSE Error:610750.149312


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
